In [81]:
import pandas as pd
from splink import DuckDBAPI, block_on, Linker, SettingsCreator
import splink.comparison_library as cl
import splink.comparison_level_library as cll
import database_processing as dbp

#asq_test_details = pd.read_sql("SELECT respondent_address_id, street_address || ' ' || city || ' ' ||  state || ' ' || postalcode as address FROM asq.asq_test_details where asq_test_filename='mock_respondent_data_upload_20250113221821_ea31eaa6.xlsx'", dbp.create_database_engine())
#respondent_address = pd.read_sql("SELECT respondent_address_id,  street_address || ' ' || city || ' ' ||  state || ' ' || postalcode as address FROM directory.respondent_address", dbp.create_database_engine())
asq_test_details = pd.read_sql("SELECT respondent_address_id as unique_id, street_address,city,state,postalcode FROM asq.asq_test_details where asq_test_filename='mock_respondent_data_upload_20250113221821_ea31eaa6.xlsx'", dbp.create_database_engine())
respondent_address = pd.read_sql("SELECT respondent_address_id as unique_id, street_address,city,state,postalcode FROM directory.respondent_address ", dbp.create_database_engine())

In [21]:
len(asq_test_details), len(respondent_address)

(2700, 107780)

In [35]:
asq_test_details

,unique_id,address
0,d6b613f7-6b6e-4dab-9990-e1a694271f32,7 CORINTHIAN C NOVATO CA 94947
1,69a25a44-961c-471e-9b8a-d011ca746c2d,32 PINE TREE CT SAN RAFAEL CA 94903
2,51fe370a-2da6-4d37-9013-c984ae68df3f,17 ALPINE RD NOVATO CA 94945
3,c5b483c9-123e-478b-ad88-323f3536921f,5 FRIENDLY LN NOVATO CA 94945
4,1c12235b-4037-4373-a860-660c24f151b7,44 PINHEIRO CI NOVATO CA 94945
...,...,...
2695,1c2b68a5-51e3-46cc-b13f-90a4773a3279,6326 Sutteridge Parkway Glendale CA 91210
2696,959bf771-7d19-4f7b-9c88-363dec758f77,5 Elka Circle Santa Monica CA 90405
2697,ecd4e25d-05e6-4dd1-a325-69d4ac7cb582,9447 Springs Avenue Sacramento CA 94280
2698,cf428a1b-c446-4451-a422-e48b228c03f4,7141 Shopko Plaza San Francisco CA 94116


In [82]:
def process_matching(test_df, directory_df):

    blocking_rules = [
        block_on("city","postalcode"),
    ]

    db_api = DuckDBAPI()

    settings = SettingsCreator(
        link_type="link_only",
        blocking_rules_to_generate_predictions=blocking_rules,
        comparisons=[
            cl.ExactMatch("postalcode"),
            cl.NameComparison("street_address"),
            cl.NameComparison("city"),
            cl.NameComparison("state"),
        ],
        retain_intermediate_calculation_columns=True,
    )
 
        
    linker = Linker([directory_df, test_df], settings, db_api=db_api, input_table_aliases=["directory", "loadfile"])

    match_probability = 0.9799999999
    max_pairs = 1e5
    model_name = 'addr_matching.json'
    session_rule =  "(state = r.state and postalcode = r.postalcode)"
    linker.training.estimate_u_using_random_sampling(max_pairs=max_pairs, seed=42)
    training_blocking_rule = block_on(session_rule)
    """
    try:
        training_session_names = (
            linker.training.estimate_parameters_using_expectation_maximisation(
            training_blocking_rule, estimate_without_term_frequencies=True
            )
        )
    except:
        training_session_names = (
            linker.training.estimate_probability_two_random_records_match(
            session_rule, recall=.7
            )
        )
    """
    try:
        df_predict = linker.inference.predict(threshold_match_probability=match_probability)
        df_e = df_predict.as_pandas_dataframe()
        linker.misc.save_model_to_json(model_name, overwrite=True)
        return df_e
    except Exception:
        print(f'Predictor failed:')
        
        df_e = pd.DataFrame()
        return df_e

In [83]:
df = process_matching(asq_test_details, respondent_address)

INFO:splink.internals.estimate_u:----- Estimating u probabilities using random sampling -----
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for city - Jaro-Winkler distance of city >= 0.92 (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for city - Jaro-Winkler distance of city >= 0.88 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for state - Jaro-Winkler distance of state >= 0.92 (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for state - Jaro-Winkler distance of state >= 0.88 (comparison vector value: 2). This usually means the comparison level was nev

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.linker_components.inference:Blocking time: 3.06 seconds


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.linker_components.inference:Predict time: 10.98 seconds
 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'postalcode':
    m values not fully trained
Comparison: 'street_address':
    m values not fully trained
Comparison: 'city':
    m values not fully trained
Comparison: 'city':
    u values not fully trained
Comparison: 'state':
    m values not fully trained
Comparison: 'state':
    u values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in your settings dictionary, or 
 - estimate with the `linker.estimate_probability_two_random_records_match` function.


In [84]:
df.to_excel('output.xlsx', index=False)

In [85]:
def load_model_and_predict(df1, df2, model_path, match_probability=0.9799999999):
    db_api = DuckDBAPI()
    
    # Load the model
    linker = Linker([df1, df2], model_path, db_api=db_api, input_table_aliases=["df1", "df2"])
    
    # Run prediction
    try:
        df_predict = linker.inference.predict(threshold_match_probability=match_probability)
        df_result = df_predict.as_pandas_dataframe()
        return df_result
    except Exception as e:
        print(f'Prediction failed: {e}')
        return pd.DataFrame()

# Example usage
model_path = 'addr_matching.json'
result_df = load_model_and_predict(asq_test_details, respondent_address, model_path)
print(result_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.linker_components.inference:Blocking time: 2.97 seconds


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.linker_components.inference:Predict time: 10.93 seconds
 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'postalcode':
    m values not fully trained
Comparison: 'street_address':
    m values not fully trained
Comparison: 'city':
    m values not fully trained
Comparison: 'city':
    u values not fully trained
Comparison: 'state':
    m values not fully trained
Comparison: 'state':
    u values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in your settings dictionary, or 
 - estimate with the `linker.estimate_probability_two_random_records_match` function.


      match_weight  match_probability source_dataset_l source_dataset_r  \
0        10.395905           0.999258              df1              df2   
1         7.886216           0.995791              df1              df2   
2         7.886216           0.995791              df1              df2   
3         6.078861           0.985422              df1              df2   
4         7.886216           0.995791              df1              df2   
...            ...                ...              ...              ...   
2571      6.932211           0.991878              df1              df2   
2572      7.517173           0.994571              df1              df2   
2573      5.932211           0.983887              df1              df2   
2574      7.517173           0.994571              df1              df2   
2575      6.517173           0.989200              df1              df2   

                               unique_id_l  \
0     f4f568bb-ee02-427f-b41c-9bcc0f81688a   
1     c

In [27]:
result_df

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,postalcode_l,postalcode_r,gamma_postalcode,bf_postalcode,...,gamma_street_address,bf_street_address,city_l,city_r,gamma_city,bf_city,state_l,state_r,gamma_state,bf_state
